Equation (4) from the Touchard paper:

$$\frac{n^2(n-1)}{6}\sigma_n = \sum_{k=1}^{n-1}(3n^2 - 10k^2)\sigma_k\sigma_{n-k}$$

So if we rearrange that equation, we can compute $\sigma_n$ for $n > 1$. The equation to compute $\sigma_n$ is:

$$\sigma_n = \frac{6}{n^2(n-1)} \sum_{k=1}^{n-1}(3n^2 - 10k^2)\sigma_k\sigma_{n-k}$$.

In [73]:
#From the email I sent you before fall last year


sigma_list = [0,1]
for n in range(2,21):
    sigma_n = 0
    for k in range(1,n):
        sigma_n += (3*(n*n) - 10*(k*k))*sigma_list[k]*sigma_list[n-k]
    sigma_list.append((6*sigma_n)//((n*n)*(n-1)))
print(sigma_list)

[0, 1, 3, 4, 7, 6, 12, 8, 15, 13, 18, 12, 28, 14, 24, 24, 31, 18, 39, 20, 42]


In [176]:
import numpy as np

In [155]:
def sigma_list(n): # To compare to the next function which uses a numpy trick
    sigma_list = [0,1] # Same thing as the last algorithm, just wrapped in a function
    for j in range(2,n+1):
        sigma_n = 0
        for k in range(1,j):
            sigma_k = sigma_list[k]
            sigma_nk = sigma_list[j-k]
            sigma_n += (3*(j*j) - 10*(k*k))*sigma_k*sigma_nk
        sigma_list.append((6*sigma_n)//((j*j)*(j-1)))
    return sigma_list

In [185]:
def sigma_vectorized(n):
    value = n
    sigmalist = np.zeros(value+1)
    sigmalist[1] = 1
    for k in range(2,value+1): #I'm sure I can use vectorization here too, just gotta figure out how
        nklist = 3*np.array([k]*(k-1))**2 - 10*np.arange(1,k)**2
        sigmalist[k] = (6/((k*k)*(k-1)))*np.dot(nklist,sigmalist[1:k]*sigmalist[k-1:0:-1])
    return sigmalist

In [179]:
sigma_list(1000)[1000]

2340

In [190]:
sigma_vectorized(1000)[1000]

2339.999999999726

In [166]:
%timeit sigma_list(1000)

157 ms ± 799 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [187]:
%timeit sigma_vectorized(1000)

41.2 ms ± 290 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


I'm thinking that maybe if I make a 2d version of `nklist` in `sigma_vectorized` with prescribed values, and just loop through slices of that instead of making a new 1d array each loop, then maybe it'd speed things up. I'll need to experiment more.